# Experiment 3.1: Basic deep learning on lying videos

* input: lying video 
* Output: Left/right leg amplitude/duration (4 scores) 

In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import tensorflow.keras as keras

from src.helpers import read_scores
from src.data_generators import RawDataGenerator
from src.data_selection import MultipleScoreSelector
from src.ai_func import cross_validation_generator
from src.settings import LYING_VIDEOS_DATA_FOLDER, SITTING_VIDEOS_DATA_FOLDER, DATA_FOLDER

In [2]:
%load_ext autoreload
%autoreload 2

### Read metadata

In [60]:
scores_df = read_scores(DATA_FOLDER / 'data_Scoring_DIS_proximal_trunk_V1.1.xlsx')

## Definitions

In [89]:
SCORES_TO_USE = ['D_RLP_R_tD_pscore', 'D_LLP_R_tD_pscore', 'D_RLP_R_tA_pscore', 'D_LLP_R_tA_pscore']
SCORERS = [1, 2, 3]

## Pipeline for training a deep neural network

### Define model architecture (here: simple CNN)

In [62]:
# Generate some dev data to get X shape
dev_selection = selector.transform(scores_df)
dev_generator = RawDataGenerator(dev_selection, videos_folder=LYING_VIDEOS_DATA_FOLDER)
X, y = dev_generator.__getitem__(0)
n_timesteps, n_features = (X.shape[1], X.shape[2])

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


In [135]:
n_outputs = len(SCORES_TO_USE)

def get_model():
    # simple CNN
    input_layer = keras.layers.Input(shape=(n_timesteps,n_features))
    norm_layer = keras.layers.BatchNormalization()(input_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(norm_layer)
    cnn_layer = keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_layer)
    cnn_layer = keras.layers.MaxPooling1D(pool_size=2)(cnn_layer)
    cnn_layer = keras.layers.Dropout(0.7)(cnn_layer)
    cnn_layer = keras.layers.Flatten()(cnn_layer)
    cnn_layer = keras.layers.Dense(100)(cnn_layer)
    output_layer = keras.layers.Dense(n_outputs)(cnn_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)
get_model().summary()

Model: "model_109"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_110 (InputLayer)       [(None, 501, 42)]         0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 501, 42)           168       
_________________________________________________________________
conv1d_436 (Conv1D)          (None, 499, 32)           4064      
_________________________________________________________________
conv1d_437 (Conv1D)          (None, 497, 32)           3104      
_________________________________________________________________
max_pooling1d_218 (MaxPoolin (None, 248, 32)           0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 248, 32)           0         
_________________________________________________________________
conv1d_438 (Conv1D)          (None, 246, 64)           62

## Train network

In [146]:
def train_model(train_generator):
    model = get_model()
    model.compile(loss='mse', optimizer=keras.optimizers.Adam())
    model.fit(train_generator, epochs=30)
    return model

def train_cross_val(cross_val):
    y_pred = []
    y_test = []
    for i_split, (train_scores, test_scores) in enumerate(cross_val):
        print(f'Fitting for 5-fold split {i_split}')
        train_generator = RawDataGenerator(train_scores, videos_folder=LYING_VIDEOS_DATA_FOLDER)
        test_generator = RawDataGenerator(test_scores, videos_folder=LYING_VIDEOS_DATA_FOLDER)
        model = train_model(train_generator)
        y_pred.append(model.predict(test_generator))
        y_test.append(test_scores)
    y_pred = np.vstack(y_pred)
    y_test = pd.concat(y_test)
    return y_test, y_pred

def evaluate(y_test, y_pred):
    results = []
    for i_score, column in enumerate(y_test):
        mae = mean_absolute_error(y_test.iloc[:, i_score], y_pred[:, i_score])
        results.append({'score': column, 'mae': mae})
    return pd.DataFrame(results)

In [147]:
results = []
for scorer in SCORERS:
    print(f'Training model for scorer {scorer}')
    selector = MultipleScoreSelector(scores_to_use=SCORES_TO_USE, scorer_to_use=scorer)
    selected_data = selector.transform(scores_df)
    cross_val = cross_validation_generator(selected_data)
    y_test, y_pred = train_cross_val(cross_val)
    results.append((y_test, y_pred))

Training model for scorer 1


/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows ['090' '058' '048'] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
49/49 [==============================] - 1s 13ms/step - loss: 2.1486
Epoch 2/30
49/49 [==============================] - 1s 13ms/step - loss: 0.4031
Epoch 3/30
49/49 [==============================] - 1s 14ms/step - loss: 0.2361
Epoch 4/30
49/49 [==============================] - 1s 13ms/step - loss: 0.1911
Epoch 5/30
49/49 [==============================] - 1s 14ms/step - loss: 0.1455
Epoch 6/30
49/49 [==============================] - 1s 14ms/step - loss: 0.1369
Epoch 7/30
49/49 [==============================] - 1s 14ms/step - loss: 0.1276
Epoch 8/30
49/49 [==============================] - 1s 13ms/step - loss: 0.0894
Epoch 9/30
49/49 [==============================] - 1s 13ms/step - loss: 0.0867
Epoch 10/30
49/49 [==============================] - 1s 13ms/step - loss: 0.0885
Epoch 11/30
49/49 [==============================] - 1s 13ms/step - loss: 0.0872
Epoch 12/30
49/49 [==============================] - 1s 14ms/step - loss: 0.0760
Epoch 13/3

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:24: UserWarning: Dropping rows ['096'] with missing video
  warnings.warn(f'Dropping rows {missing_ids} with missing video')
/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
37/37 [==============================] - 2s 13ms/step - loss: 5.4524
Epoch 2/30
37/37 [==============================] - 1s 13ms/step - loss: 1.2961
Epoch 3/30
37/37 [==============================] - 1s 13ms/step - loss: 0.5143
Epoch 4/30
37/37 [==============================] - 0s 13ms/step - loss: 0.2872
Epoch 5/30
37/37 [==============================] - 1s 14ms/step - loss: 0.2556
Epoch 6/30
37/37 [==============================] - 1s 14ms/step - loss: 0.2347
Epoch 7/30
37/37 [==============================] - 1s 13ms/step - loss: 0.1499
Epoch 8/30
37/37 [==============================] - 1s 13ms/step - loss: 0.1637
Epoch 9/30
37/37 [==============================] - 0s 13ms/step - loss: 0.1361
Epoch 10/30
37/37 [==============================] - 1s 13ms/step - loss: 0.1113
Epoch 11/30
37/37 [==============================] - 0s 13ms/step - loss: 0.1127
Epoch 12/30
37/37 [==============================] - 1s 14ms/step - loss: 0.1174
Epoch 13/3

/Users/svenvanderburg/projects/modys-video/src/data_selection.py:34: UserWarning: Dropping rows [] with missing score
  warnings.warn(f'Dropping rows {only_na.index.values} with missing score')


Fitting for 5-fold split 0
Epoch 1/30
36/36 [==============================] - 1s 17ms/step - loss: 5.2329
Epoch 2/30
36/36 [==============================] - 1s 16ms/step - loss: 0.6981
Epoch 3/30
36/36 [==============================] - 1s 15ms/step - loss: 0.5101
Epoch 4/30
36/36 [==============================] - 0s 13ms/step - loss: 0.2743
Epoch 5/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1776
Epoch 6/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1644
Epoch 7/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1307
Epoch 8/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1487
Epoch 9/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1434
Epoch 10/30
36/36 [==============================] - 0s 13ms/step - loss: 0.1013
Epoch 11/30
36/36 [==============================] - 0s 13ms/step - loss: 0.0972
Epoch 12/30
36/36 [==============================] - 0s 13ms/step - loss: 0.0994
Epoch 13/3

In [148]:
for scorer, (y_test, y_pred) in zip(SCORERS, results):
    print(f'results for scorer {scorer}')
    print(evaluate(y_test, y_pred))

results for scorer 1
               score       mae
0  D_RLP_R_tD_pscore  0.335920
1  D_LLP_R_tD_pscore  0.379423
2  D_RLP_R_tA_pscore  0.261640
3  D_LLP_R_tA_pscore  0.302673
results for scorer 2
               score       mae
0  D_RLP_R_tD_pscore  0.342937
1  D_LLP_R_tD_pscore  0.377326
2  D_RLP_R_tA_pscore  0.306030
3  D_LLP_R_tA_pscore  0.276037
results for scorer 3
               score       mae
0  D_RLP_R_tD_pscore  0.605735
1  D_LLP_R_tD_pscore  0.614131
2  D_RLP_R_tA_pscore  0.555853
3  D_LLP_R_tA_pscore  0.567253
